In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("LANGCHAIN_TRACING_V2")

In [3]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal
from langchain_core.tools import tool


@tool
def get_weather(city):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]


# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools)

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
inputs = {"messages": [("user", "what is the weather in chongqing , China?")]}
from langchain_core.tracers.context import tracing_v2_enabled
with tracing_v2_enabled(project_name="My test Project"):
    print_stream(graph.stream(inputs, stream_mode="values"))
inputs = {"messages": [("user", "who built you?")]}
print_stream(graph.stream(inputs, stream_mode="values"))
from langchain_core.tools import tool



In [ ]:

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b



# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)


In [ ]:

import getpass
import os
from langchain_openai import ChatOpenAI
from tool import find_anchors

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

# First we initialize the model we want to use.

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal
from langchain_core.tools import tool


tools = [find_anchors]

# We can add "chat memory" to the graph with LangGraph's checkpointer
# to retain the chat context between interactions
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# system prompt
sys_prompt = "You are a professional code consultant. You will help me understand the community structure of my repository code graph."
# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools,interrupt_before=["tools"] , checkpointer=memory,state_modifier=sys_prompt)

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
            
            
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [("user", "What tool can u use? please call it on next step ,label is module A")]}

print_stream(graph.stream(inputs, config=config, stream_mode="values"))

In [ ]:
snapshot = graph.get_state(config)
print("Next step: ", snapshot.next)

In [ ]:
print_stream(graph.stream(None, config, stream_mode="values"))